In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Function
import numpy as np
from torchvision.models import resnet18

In [14]:
def pairwise_distance(x1, x2):
    diff = torch.abs(x1 - x2)
    return torch.pow(diff, 2).sum(dim=1)

In [16]:
class TripletLoss(Function):
    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
            positive_dist = pairwise_distance(anchor, positive)
            negative_dist = pairwise_distance(anchor, negative)
            loss = positive_dist - negative_dist + self.margin
            relu = nn.ReLU()
            loss_final = relu(loss).mean()
            return loss_final


def triplet_loss(anchor, positive, negative, margin=0.2):
    t_loss = TripletLoss(margin)
    loss = t_loss(anchor, positive, negative)
    return loss

In [17]:
class FaceNet(nn.Module):
    def __init__(self, embedding_dimensions=64):
        self.embedding_dimensions = embedding_dimensions
        self.model = resnet18(pretrained = True)
        resnet_fc_in = self.model.fc.in_features
        self.model.fc = nn.Linear(resnet_fc_in, embedding_dimensions)

        #Initialize weights
        self.model.fc.weight.data.normal_(0.0, 0.02)
        self.model.fc.bias.data.fill_(0)

        self.minibatch_size = 10
        self.lr = 1e-4
        self.loss_fn = triplet_loss
        optimizer = torch.optim.Adam(self.model.parameters(), lr = self.lr)

    def forward(self, input_images):
        output = self.model(input_images)
        return output

    def train(self):
        total_epochs = 50000
        last_saved_epoch = 0
        for epoch in range(last_saved_epoch, num_epochs):
            self.model.train()
            pass
    